In [112]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams
import warnings

Reading Training File

In [113]:
train_data=pd.read_csv('../input/store-sales-time-series-forecasting/train.csv')
train_data.head()

In [144]:
#Dropping id column
train_data=train_data.drop(['id'],axis=1)

Reading other supplementary file

In [145]:
stores=pd.read_csv('../input/store-sales-time-series-forecasting/stores.csv')
holidays=pd.read_csv('../input/store-sales-time-series-forecasting/holidays_events.csv')
oil=pd.read_csv('../input/store-sales-time-series-forecasting/oil.csv')
transactions = pd.read_csv('../input/store-sales-time-series-forecasting/transactions.csv')

In [146]:
train_data['date']=pd.to_datetime(train_data['date'])
train_data.head()

Merging store data with transaction table

In [147]:
store_data=pd.merge(train_data,stores,on='store_nbr',how='left')
store_data.head()

Average unit sales for each state

In [148]:
state_sales = store_data.groupby(['state'], as_index=False).sales.sum()
state_sales = state_sales.sort_values(by='sales',ascending=False)

In [149]:
state_sales.head()

In [150]:
sns.set(rc={'figure.figsize':(18,5)})
ax = sns.barplot(x="state", y="sales", data=state_sales)
plt.title('Total sales per state')
plt.xlabel('State')
plt.ylabel('Total Sales')
plt.xticks(rotation='60')
plt.show()

From the above data, we can infer that the city of Pichincho contributes for very amount of sales

City wise sales Analysis

In [151]:
city_sales = store_data.groupby(['city'],as_index=False)['sales'].sum()
city_sales=city_sales.sort_values(by='sales',ascending=False)

In [152]:
city_sales.head()

In [153]:
sns.set(rc={'figure.figsize':(18,5)})
ax = sns.barplot(x="city", y="sales", data=city_sales)
plt.title('Total sales per City')
plt.xlabel('City')
plt.ylabel('Total Sales')
plt.xticks(rotation='90')
plt.show()

In [154]:
store_data[store_data['city']=='Quito']

From the above chart, max of products are sold from store loacted at Quito

Sales per Store

In [155]:
#Total unit sales per store
store_sales=store_data.groupby(['store_nbr'],as_index=False)['sales'].sum()
store_sales = store_sales.sort_values(by='sales',ascending=False)

In [156]:
store_sales.head()

In [157]:
sns.set(rc={'figure.figsize':(18,5)})
ax = sns.barplot(x="store_nbr", y="sales", data=store_sales)
plt.title('Total sales per City')
plt.xlabel('Store Number')
plt.ylabel('Total Sales')
plt.xticks(rotation='90')
plt.show()

In [158]:
#Checking which state generates the highest sales
store_data[(store_data['store_nbr']>=44) & (store_data['store_nbr']<50)]

In [159]:
pichincha_data=store_data[store_data['state']=='Pichincha']
pichincha_data.shape

In [160]:
pichincha_data['store_nbr'].value_counts()

In [161]:
pichincha_data_percent=(len(pichincha_data)/len(train_data))*100
pichincha_data_percent

From above, high sales volume 35% of the data is from the state Pinchincha so dropping this state data

In [162]:
store_data=store_data[store_data['state']!='Pichincha']
store_data.shape

Sales per unit Family of products

In [163]:
family_sales=store_data.groupby(['family'],as_index=False)['sales'].sum()
family_sales=family_sales.sort_values(by='sales',ascending=False)

sns.set(rc={'figure.figsize':(18,5)})
ax = sns.barplot(x="family", y="sales", data=family_sales)
plt.title('Total sales per Family of Products')
plt.xlabel('Family')
plt.ylabel('Total Sales')
plt.xticks(rotation='90')
plt.show()

In [164]:
family_sales['family']

Replacing the long tails of data with Family 'OTHERS'

In [165]:
store_data['family']=store_data['family'].replace(['HOME AND KITCHEN I', 'GROCERY II', 'SEAFOOD', 
                                                   'HOME AND KITCHEN II', 'CELEBRATION', 'LINGERIE', 'AUTOMOTIVE', 'LAWN AND GARDEN', 
                                                   'PLAYERS AND ELECTRONICS', 'LADIESWEAR', 'BEAUTY', 'PET SUPPLIES', 'MAGAZINES',
                                                   'SCHOOL AND OFFICE SUPPLIES', 'HARDWARE', 'HOME APPLIANCES', 'BABY CARE', 'BOOKS'],'OTHERS')

In [166]:
family_sales=store_data.groupby(['family'],as_index=False)['sales'].sum()
family_sales=family_sales.sort_values(by='sales',ascending=False)

sns.set(rc={'figure.figsize':(12,4)})
ax = sns.barplot(x="family", y="sales", data=family_sales)
plt.title('Total sales per Family of Products')
plt.xlabel('Product Family')
plt.ylabel('Total Sales')
plt.xticks(rotation='90')
plt.show()

In [167]:
store_data.shape

Sales per Cluster

In [168]:
cluster_sales=store_data.groupby(['cluster'],as_index=False)['sales'].sum()
cluster_sales=cluster_sales.sort_values(by='sales',ascending=False)

sns.set(rc={'figure.figsize':(10,4)})
ax = sns.barplot(x="cluster", y="sales", data=cluster_sales)
plt.title('Total sales per Cluster')
plt.xlabel('Cluster')
plt.ylabel('Total Sales')
plt.show()

In [169]:
type_sales=store_data.groupby(['type'],as_index=False)['sales'].sum()
type_sales=type_sales.sort_values(by='sales',ascending=False)

sns.set(rc={'figure.figsize':(10,4)})
ax = sns.barplot(x='type', y="sales", data=type_sales)
plt.title('Total sales per Type')
plt.xlabel('Type')
plt.ylabel('Total Sales')
plt.show()

In [170]:
#dropping column city as we are already having 'State' as a feature
store_data=store_data.drop(['city'],axis=1)

In [171]:
store_data.head()

**Merging oil and holidays data**

In [172]:
oil.head()

In [179]:
oil['date']=pd.to_datetime(oil['date'])
oil=oil.rename(columns={'dcoilwtico':'oil_price'})
store_oil_data=pd.merge(store_data,oil,on='date',how='left')
store_oil_data.head()

In [186]:
#Adding holiday data to the store+oil dataframe
holidays['type'].unique()

#Replacing the holiday type with only 'Holiday'
holidays['type']=holidays['type'].replace(['Transfer','Additional','Bridge','Event'],'Holiday')

#Rest of columns of holiday is not required so dropping them completely
holidays=holidays.drop(['locale','locale_name','description','transferred'],axis=1)

In [187]:
holidays.head()

In [188]:
#Merging holidays with store+oil data
holidays['date']=pd.to_datetime(holidays['date'])
holidays=holidays.rename(columns={'type':'day_type'})

final_data=pd.merge(store_oil_data,holidays,on='date',how='left')
final_data.head()

In [191]:
#Splitting date into day,month,year 
final_data['day']=final_data['date'].dt.day
final_data['month']=final_data['date'].dt.month
final_data['year']=final_data['date'].dt.year
final_data.head()

In [192]:
final_data['quarter']=final_data['date'].dt.quarter

In [196]:
final_data.isnull().sum().sort_values(ascending=False)

There are 2 columns with Null values and below is how it is imputed
1. day_type - We treat each day_type entry to be working for all for the missing values
2. oil_price - For the missing values we will populate the prev day data

In [197]:
final_data['day_type']=final_data['day_type'].fillna('Work Day')
final_data['oil_price']=final_data['oil_price'].fillna(axis=0,method='ffill')
final_data.isnull().sum().sort_values(ascending=False)

In [198]:
final_data.info()

**Yearly Sales**

In [203]:
final_data.year.value_counts()

In [204]:
final_data_subset = final_data[final_data['year']!='2017']

In [211]:
yearly_sales=final_data_subset.groupby(['year'],as_index=False)['sales'].mean()
yearly_sales=yearly_sales.sort_values(by='sales',ascending=False)

sns.set(rc={'figure.figsize':(8,4)})
ax = sns.barplot(x='year', y="sales", data=yearly_sales)
plt.title('Avg Yearly Sales')
plt.xlabel('Year')
plt.ylabel('Total Sales')
plt.show()

**Monthly Sales**

In [214]:
monthly_sales=final_data_subset.groupby(['month'],as_index=False)['sales'].mean()
monthly_sales=monthly_sales.sort_values(by='sales',ascending=False)

sns.set(rc={'figure.figsize':(8,4)})
ax = sns.barplot(x='month', y="sales", data=monthly_sales)
plt.title('Avg Monthly Sales')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.show()

**Quaterly Sales**

In [216]:
quarterly_sales=final_data_subset.groupby(['quarter'],as_index=False)['sales'].mean()
quarterly_sales=quarterly_sales.sort_values(by='sales',ascending=False)

sns.set(rc={'figure.figsize':(8,4)})
ax = sns.barplot(x='quarter', y="sales", data=quarterly_sales)
plt.title('Avg Quarterly Sales')
plt.xlabel('Quater')
plt.ylabel('Total Sales')
plt.show()

**Daily Avg Sales**

In [223]:
daily_sales=final_data_subset.groupby(['day'],as_index=False)['sales'].mean()

sns.set(rc={'figure.figsize':(10,4)})
ax = sns.barplot(x='day', y="sales", data=daily_sales)
plt.title('Avg Sales per Day')
plt.xlabel('Day')
plt.ylabel('Total Sales')
plt.show()

How oil prices affect unit sales

In [230]:
oil_sales=final_data[['date','oil_price','sales']]
d={'oil_price':'oil_price','sales':'total_sales'}
oil_sales=oil_sales.groupby(['date']).agg({'oil_price':'mean','sales':'sum'}).rename(columns=d)
oil_sales.head()

In [233]:
plt.scatter(oil_sales.oil_price,oil_sales.total_sales)
plt.xlabel('Oil Price')
plt.ylabel('Sales')
plt.title('Variation of Sales with Oil Prices')
plt.show()

From above we see as the oil price increases from 40 to 60 there is a dip in total sales

In [248]:
holiday_sales=final_data[['day_type','sales']]
holiday_sales=holiday_sales.groupby(['day_type'],as_index=False).agg({'sales':'sum'})
holiday_sales

In [250]:
sns.set(rc={'figure.figsize':(10,4)})
ax = sns.barplot(x='day_type', y="sales", data=holiday_sales)
plt.title('Variation of Sales Based on Holidays')
plt.xlabel('Day Type')
plt.ylabel('Total Sales')
plt.show()

Sales are low on holidays as compared to sales on working days